In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Read MySQL Data") \
    .config("spark.jars", "C:\Users\prana\OneDrive\Desktop\AI Sem 1\Big Data Tools 2\mysql-connector-java-8.0.26.jar") \
    .getOrCreate()



In [ ]:
mysql_url = "jdbc:mysql://localhost:3306/your_database"
mysql_properties = {
    "user": "root",
    "password": "mypassword",
    "driver": "com.mysql.cj.jdbc.Driver"
}


In [ ]:
df = spark.read.jdbc(url=mysql_url, table="your_table", properties=mysql_properties)


In [4]:
from pyspark.sql.functions import col, to_date, regexp_replace

# Extract the date part before the second comma
df = df.withColumn("Date", regexp_replace(col("Date"), r", [A-Za-z]+$", ""))

# Convert the Date column to DateType
df = df.withColumn("Date", to_date(col("Date"), "MMM dd, yyyy"))

# Reformat the date to yyyy-MM-dd
df = df.withColumn("Date", col("Date").cast("string"))


In [5]:
from pyspark.sql.functions import monotonically_increasing_id

# Add a row number column
df = df.withColumn("RowNumber", monotonically_increasing_id())

# Select and rename the required columns
df = df.select(col("RowNumber"), col("Title"), col("Date"))

# Show the resulting DataFrame
df.show(truncate=False)



+---------+---------------------------------------------------------------------------+----------+
|RowNumber|Title                                                                      |Date      |
+---------+---------------------------------------------------------------------------+----------+
|0        |ATMs to become virtual bank branches, accept deposits with instant credit  |2020-05-26|
|1        |IDFC First Bank seniors to forgo 65% of bonus amid Covid-19 crisis         |2020-05-26|
|2        |Huge scam in YES Bank for many years, says Enforcement Directorate         |2020-05-25|
|3        |Bank of Maharashtra sanctioned Rs 2,789 cr in loans to MSMEs in 3 months   |2020-05-24|
|4        |DCB Bank's profit before tax declines 37.6% to Rs 93.84 crore in Q4        |2020-05-23|
|5        |Bank of Baroda to advance Rs 12,000 crore to MSMEs under govt scheme       |2020-05-23|
|6        |Retail, MSME loans to get cheaper as RBI cuts repo rate in surprise action |2020-05-22|
|7        

In [ ]:
azure_sql_url = "jdbc:sqlserver://alinizarserver1.database.windows.net:1433;database=myfirstdatabase;"
azure_sql_properties = {
    "user": "azureuser@alinizarserver1",
    "password": "Ilove85workWonder69",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

df = spark.read.jdbc(url=azure_sql_url, table="IndianFinancialArticles", properties=azure_sql_properties)

In [ ]:
# Write DataFrame to Azure SQL Database
df.write.jdbc(url=azure_sql_url, table="IndianFinancialArticles", mode="overwrite", properties=azure_sql_properties)